In [1]:
import robotic as ry
import numpy as np
import time

In [2]:
config = ry.Config()
config.addFile("puzzles/p5-wall-easy.g")

In [3]:
DEBUG_MODE = False

In [4]:
EGO_NAME = "ego"
OBJ_NAME = "obj"
SUB_GOAL_NAME = "sub-goal1"
GOAL_NAME = "goal"
CAMERA_NAME = "camera_top"
FLOOR_NAME = "floor"

GAP = config.getFrame(EGO_NAME).getSize()[1] * 2

In [5]:
config.view()
f = config.getFrame(CAMERA_NAME)
config.view_setCamera(f)

In [6]:
q_start = config.getFrame(EGO_NAME).getPosition()[:2]
q_obj = config.getFrame(OBJ_NAME).getPosition()[:2]
q_goal = config.getFrame(GOAL_NAME).getPosition()[:2]

In [7]:
def is_in_line_of_sight(position : np.array, target_frame : ry.Frame, config : ry.Config) -> bool:
    frame_position = target_frame.getPosition()
    frame_size = target_frame.getSize()
    frame_hypotenuse = np.sqrt(frame_size[0]**2 + frame_size[1]**2)

    sight_ray = frame_position[:2]  - position
    alpha = np.arctan2(sight_ray[1], sight_ray[0])
    theta = abs(alpha)

    offset = frame_hypotenuse / 2 # can be made more precise

    box_length = np.sqrt(sight_ray[0]**2 + sight_ray[1]**2) - 2 * offset
    box_width = frame_hypotenuse * max(
        np.cos(np.pi/4 - theta),
        np.sin(theta - np.pi/4)
    )
    box_height = .2

    box_x = position[0] + sight_ray[0]/2
    box_y = position[1] + sight_ray[1]/2
    box_z = 0.3
    box_yaw = alpha

    copy_config = ry.Config()
    copy_config.addConfigurationCopy(config)
    copy_config.addFrame("line_of_sight") \
        .setShape(ry.ST.box, [box_length, box_width, box_height]) \
        .setContact(1) \
        .setColor([1, 1, 1]) \
        .setPose(f"t({box_x} {box_y} {box_z}) E(0 0 {box_yaw})")
    copy_config.addFrame("point_A") \
        .setShape(ry.ST.marker, [.2]) \
        .setPosition(frame_position)
    copy_config.addFrame("point_B") \
        .setShape(ry.ST.marker, [.2]) \
        .setPosition(list(position) + [0.3])
    
    if DEBUG_MODE:
        copy_config.view_setCamera(copy_config.getFrame('camera_top'))
        copy_config.view(pause=True, message=f"box_width: {box_width:.2f}, alpha: {alpha/np.pi*180}, line: {sight_ray}")
        copy_config.view_close()

    is_in_sight = copy_config.getCollisionFree()
    del copy_config
    return is_in_sight

In [8]:
def cartesian_to_polar(coordinates):
    x = coordinates[0]
    y = coordinates[1]
    r = np.hypot(x, y)
    theta = np.arctan2(y, x)
    return r, theta

In [9]:
def polar_to_cartesian(coordinates):
    r = coordinates[0]
    theta = coordinates[1]
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    return x, y

In [10]:
def solve_touch_with_komo(komo_config):
    komo = ry.KOMO(komo_config, phases=1, slicesPerPhase=1, kOrder=0, enableCollisions=True)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, scale=[1e1])
    komo.addObjective([], ry.FS.negDistance, [EGO_NAME, OBJ_NAME], ry.OT.eq, [1e1])
    ret = ry.NLP_Solver(komo.nlp(), verbose=1).solve()
    if ret:
        path = komo.getPath()
        return path
    else:
        return None

In [11]:
ry.params_clear()
ry.params_add({
    "rrt/stepsize": 0.05,
    "rrt/verbose": 1
})

def solve_rrt(cur_config : ry.Config, q_begin : np.array, q_object : np.array, q_target : np.array):
    ITERATIONS = 24
    delta_theta = 2 * np.pi / ITERATIONS
    for i in range(ITERATIONS):
        transpose_polar1 = np.array([GAP, i * delta_theta])
        transpose_cartesian1 = np.array(polar_to_cartesian(transpose_polar1))
        q_home1 = q_object + transpose_cartesian1
        cur_config.setJointState(q_home1)
        q_sub = solve_touch_with_komo(cur_config)
        if q_sub is not None:
            cur_config.setJointState(q_begin)
            rrt = ry.PathFinder()
            rrt.setProblem(cur_config, [q_begin], [q_sub])
            solution1 = rrt.solve()
            del rrt
            
            cur_config.setJointState(q_sub)
            cur_config.attach(EGO_NAME, OBJ_NAME)
            q_real_goal = q_target + q_sub - q_object

            rrt = ry.PathFinder()
            rrt.setProblem(cur_config, [q_sub], [q_real_goal])
            solution2 = rrt.solve()
            cur_config.attach(FLOOR_NAME, OBJ_NAME)
            cur_config.setJointState(q_begin)
            cur_config.view()
            if solution1.feasible and solution2.feasible:
                return solution1, solution2

    cur_config.setJointState(q_begin)            
    return None, None

In [12]:
def get_proximity_score(position : np.array, goal_frame : ry.Frame) -> int:
    distance = np.linalg.norm(position - goal_frame.getPosition()[:2])
    if distance < 0.2:
        return 5
    elif distance < 0.4:
        return 2
    else:
        return 0

In [13]:
def get_subgoal_score(
        subgoal_position : np.array,
        agent_frame : ry.Frame,
        goal_frame : ry.Frame,
        config : ry.Config) -> int:
    return 10 * int(is_in_line_of_sight(subgoal_position, goal_frame, config)) \
        + 5 * int(is_in_line_of_sight(subgoal_position, agent_frame, config)) \
        + get_proximity_score(subgoal_position, goal_frame)

In [20]:
def select_node(configs : list) -> ry.Config:
    best_score = 0
    config_index = 0
    best_config = configs[config_index]
    for i in range(len(configs)):
        sample_config = configs[i]
        score = get_subgoal_score(sample_config.getFrame(OBJ_NAME).getPosition()[:2], 
                                  sample_config.getFrame(EGO_NAME),
                                  sample_config.getFrame(GOAL_NAME),
                                  sample_config)
        if score > best_score:
            best_score = score
            best_config = sample_config
            config_index = i
    return best_config, config_index

In [ ]:
def solve(config : ry.Config, goal_frame : ry.Frame) -> list:
    # TODO implement solve function
    return None

In [ ]:
# return the path to the goal
def sub_solve(config : ry.Config, goal_frame : ry.Frame) -> list:
    # TODO implement sub_solve function
    return None

In [15]:
def reachable(reach_config : ry.Config, object_frame : ry.Frame) -> bool:
    copy_config = ry.Config()
    copy_config.addConfigurationCopy(reach_config)
    copy_config.getFrame(OBJ_NAME).setContact(0)
    rrt = ry.PathFinder()
    rrt.setProblem(copy_config, [copy_config.getJointState()], [object_frame.getPosition[:2]])
    ret = rrt.solve()
    del rrt
    del copy_config
    return ret.feasible

In [16]:
POINT_COUNT = 200
THRESHOLD = 5
SUBSET_SIZE = 20
def propose_subgoals(config : ry.Config, object_frame : ry.Frame) -> list:
    generated_points = np.random.uniform(low=-2, high=2, size=(POINT_COUNT, 2)) # random point sampling

    filtered_points = []
    for point in generated_points:
        config.getFrame(SUB_GOAL_NAME).setPosition([point[0], point[1], 0.15])
        score = get_subgoal_score(point, config.getFrame(EGO_NAME), config.getFrame(GOAL_NAME), config)
        if score >= THRESHOLD:
            filtered_points.append(point)

    points_array = np.array(filtered_points)
    subset_size = min(len(filtered_points), SUBSET_SIZE)
    point_subset = points_array[np.random.choice(points_array.shape[0], size=subset_size, replace=False)]
    point_subset += object_frame.getPosition()[:2]
    return point_subset

In [ ]:
subgoals = propose_subgoals(config, config.getFrame(OBJ_NAME))
count = 0
start_configs = []
for subgoal in subgoals:
    count+=1
    subgoal_position = config.getFrame(SUB_GOAL_NAME).getPosition()
    subgoal_position[:2] = subgoal
    config.getFrame(SUB_GOAL_NAME).setPosition(subgoal_position)
    solution1, solution2 = solve_rrt(config, q_start, q_obj, subgoal)
    if solution1 is not None:
        start_configs.append([solution1, solution2])
    
print(f"available subgoals: {len(start_configs)}")

In [ ]:
final_solution = []
for initial_config in start_configs:
    copy_config = ry.Config()
    copy_config.addConfigurationCopy(config)
    copy_config.setJointState(initial_config[0].x[-1])
    copy_config.attach(EGO_NAME, OBJ_NAME)
    copy_config.setJointState(initial_config[1].x[-1])
    copy_config.attach(FLOOR_NAME, OBJ_NAME)
    solution1, solution2 = solve_rrt(copy_config,
                copy_config.getJointState(),
                copy_config.getFrame(OBJ_NAME).getPosition()[:2],
                q_goal)
    if solution1 is not None:
        final_solution.extend([initial_config[0], initial_config[1], solution1, solution2])
        del copy_config
        break
    del copy_config

if len(final_solution) == 0:
    print("No solution found")


In [17]:
def animate_solution(anim_config, solution):
    for state in solution.x:
        anim_config.setJointState(state)
        anim_config.view()
        time.sleep(0.025)

In [ ]:
parity = True
for step in final_solution:
    animate_solution(config, step)
    if parity:
        config.attach(EGO_NAME, OBJ_NAME)
    else:
        config.attach(FLOOR_NAME, OBJ_NAME)
    parity = not parity

In [22]:
# forward subproblem search algorithm
objects = [OBJ_NAME]
def forward_subproblem_search(start_config : ry.Config, goal_frame : ry.Frame):
    L = [start_config]
    while len(L) > 0:
        x, index = select_node(L)
        L.pop(index)
        solution1, solution2 = solve_rrt(x,
                                         x.getJointState(),
                                         x.getFrame(OBJ_NAME).getPosition()[:2],
                                         goal_frame.getPosition()[:2])
        if solution1 is not None: # if found
            return x, solution1, solution2
        
        for object in objects: # there is just one object for now
            object_frame = x.getFrame(object)
            if not reachable(x, object_frame):
                continue
            subgoals = propose_subgoals(x, object_frame)
            for subgoal in subgoals:
                new_config = ry.Config()
                new_config.addConfigurationCopy(x)
                new_config.getFrame(SUB_GOAL_NAME).setPosition(subgoal + [0.15])
                path1, path2 = solve_rrt(new_config, new_config.getJointState(), object_frame.getPosition()[:2], subgoal)
                # if path found and not rej(L, x^f) then
                if path1 is not None:
                    new_config.setJointState(path1.x[-1])
                    new_config.attach(EGO_NAME, OBJ_NAME)
                    new_config.setJointState(path2.x[-1])
                    new_config.attach(FLOOR_NAME, OBJ_NAME)
                    L.append(new_config) # add last elememnt of path to L
    return None, None, None

In [23]:
solution_config, path1, path2 = forward_subproblem_search(config, config.getFrame(GOAL_NAME))
if solution_config is not None:
    animate_solution(config, path1)
    config.attach(EGO_NAME, OBJ_NAME)
    animate_solution(config, path2)
    config.attach(FLOOR_NAME, OBJ_NAME)

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1 	lambda:[]
==nlp== it:   0  evals:   2  A(x):6.82555e-06  f:          0  g:          0  h: 0.00261257  |x-x'|:  0.0522515 	stop:DeltaConverge 	x:[-0.549739, -1.50968]
==nlp== it:   1  evals:   2  A(x):4.77788e-05  mu:5 	lambda:[0, -0.00522515]
==nlp== it:   1  evals:   2  A(x):4.77788e-05  f:          0  g:          0  h: 0.00261257  |x-x'|:          0 	stop:DeltaConverge 	x:[-0.549739, -1.50968]
==nlp== StoppingCriterion Delta<0.01
  -- rrt success: queries:2361 tree sizes: 247 146
  -- rrt success: queries:6272 tree sizes: 129 385


In [24]:
config.view_close()
del config